In [1]:
import gridthings

text = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""

grid = gridthings.Grid(text)
grid

<Grid shape=(10, 10)>

In [2]:
collection = []
current = []
# use grid.data.values() instead of grid.flatten() to avoid "wrap-around" numbers, e.g.
# . . 3
# 4 & .
for row in grid.data.values():
    for c in row.values():
        if c.value.isdigit():
            current.append(c)
        else:
            if current:
                collection.append(current)
                current = []
    # end of row
    if current:
        collection.append(current)
        current = []

collection

[[Cell(y=0, x=0, value='4'),
  Cell(y=0, x=1, value='6'),
  Cell(y=0, x=2, value='7')],
 [Cell(y=0, x=5, value='1'),
  Cell(y=0, x=6, value='1'),
  Cell(y=0, x=7, value='4')],
 [Cell(y=2, x=2, value='3'), Cell(y=2, x=3, value='5')],
 [Cell(y=2, x=6, value='6'),
  Cell(y=2, x=7, value='3'),
  Cell(y=2, x=8, value='3')],
 [Cell(y=4, x=0, value='6'),
  Cell(y=4, x=1, value='1'),
  Cell(y=4, x=2, value='7')],
 [Cell(y=5, x=7, value='5'), Cell(y=5, x=8, value='8')],
 [Cell(y=6, x=2, value='5'),
  Cell(y=6, x=3, value='9'),
  Cell(y=6, x=4, value='2')],
 [Cell(y=7, x=6, value='7'),
  Cell(y=7, x=7, value='5'),
  Cell(y=7, x=8, value='5')],
 [Cell(y=9, x=1, value='6'),
  Cell(y=9, x=2, value='6'),
  Cell(y=9, x=3, value='4')],
 [Cell(y=9, x=5, value='5'),
  Cell(y=9, x=6, value='9'),
  Cell(y=9, x=7, value='8')]]

In [3]:
from dataclasses import dataclass
from typing import List
from gridthings import Cell, OutOfBoundsCell

@dataclass
class Number:
    cells: List[Cell]

    @property
    def value(self) -> int:
        s = ''.join(c.value for c in self.cells)
        return int(s)

    def gears(self) -> List[Cell]:
        "Return the list of Gears (*) touching this Number"
        return [item for item in self.peek_all() if item.value == "*"]

    def peek_all(self) -> List[Cell]:
        """
        Returns all adjacent cells to this Number, excluding cells belonging to this
        Number group or out of bounds cells
        """
        total_results = []
        for cell in self.cells:
            results = grid.peek_all(cell.y, cell.x)
            for item in results:
                if item in self.cells:
                    continue
                if isinstance(item, OutOfBoundsCell):
                    continue
                if item in total_results:
                    continue
                total_results.append(item)
        return total_results

    def __repr__(self):
        return f"<Number {self.value})>"

nums = []
for item in collection:
    nums.append(Number(cells=item))

nums

[<Number 467)>,
 <Number 114)>,
 <Number 35)>,
 <Number 633)>,
 <Number 617)>,
 <Number 58)>,
 <Number 592)>,
 <Number 755)>,
 <Number 664)>,
 <Number 598)>]

In [4]:
nums[0].gears()

[Cell(y=1, x=3, value='*')]

In [5]:
import collections
gear_counts = collections.defaultdict(list)

for n in nums:
    for gear in n.gears():
        gear_counts[gear].append(n)

gear_counts
    

defaultdict(list,
            {Cell(y=1, x=3, value='*'): [<Number 467)>, <Number 35)>],
             Cell(y=4, x=3, value='*'): [<Number 617)>],
             Cell(y=8, x=5, value='*'): [<Number 755)>, <Number 598)>]})

In [6]:
answer = 0
for c, matches in gear_counts.items():
    if len(matches) == 2:
        gear_ratio = matches[0].value * matches[1].value
        answer += gear_ratio
answer

467835